# (저와 같은) 초보 중에서도 초보를 위한 베이스라인

### 저와 같은 초보 중에서도 초보들이 저처럼 시간낭비?를 하지 않기를 바라는 마음으로 이렇게 코드공유를 해봅니다.

####! 이미 한번이라도 제출해본 적이 있으신 분은 안보고 넘어가셔도 좋습니다 !



이 코드 공유는 아래의 어려움을 겪으신 분들을 위한 베이스라인? 코드입니다.
- 데이콘이라는걸 처음 알게 된 분들이 **경험삼아 한번이라도 제출하고 싶은 경우**
- 베이스라인대로 코드를 돌려봤는데 데이터프레임 칼럼명(헤더)가 깨져서 당황하신 분들
- 제공되는 기본 베이스라인코드를 그대로 돌려봤는데 본인이 코드 실행해서 만들어진 **submission.csv와 sample_submission.csv가 서로 다르게 생겨서** 당황하신 분들
- sklearn에서 **GridSearchCV**를 써본적이 없는 분들
- **SMAPE**를 어떻게 loss함수로 지정해야하는지 몰라 포기하셨던 분들



거의 대다수의 분들은 위의 사항에 해당되지 않음을 알고있습니다...만 그래도 저같이 판다스 람다함수 같은 기본적인 것도 못 외워서 검색하고 어디를 실수했는지 못찾아서 거의 하루를 매달려야 하는 분들도 있기는 있을 것이라고 생각하여 올려봅니다


(주의) 기존 베이스라인코드를 많이 베껴왔습니다.

# 데이터 전처리

In [1]:
#Library Imports
import numpy as np  # 넘파이
import pandas as pd  # 판다스
# import math
import os  # 디렉토리 변경
from sklearn.model_selection import GridSearchCV  # 파라미터 설정 고민을 줄여주는 고마운 친구
from sklearn.metrics import make_scorer  # loss function 커스터마이징

from lightgbm import LGBMRegressor  # 전 LGBMRegressor 만을 사용해서 돌려보았습니다.
from sklearn.model_selection import train_test_split  # train, valid set 제작

os.chdir('/content/drive/MyDrive/Colab Notebooks/데이콘/전력사용량 예측') 

In [2]:
# 데이터 로드 (인코딩은 euc-kr)
train=pd.read_csv('train.csv', encoding='euc-kr')
test=pd.read_csv('test.csv', encoding='euc-kr')
submission=pd.read_csv('sample_submission.csv', encoding='euc-kr')

In [3]:
train.head(7)

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
5,1,2020-06-01 05,8010.576,16.9,3.4,93.0,0.0,0.0,0.0,0.0
6,1,2020-06-01 06,7978.176,16.7,3.4,90.0,0.1,0.0,0.0,0.0


In [4]:
test.head(7)  # 전력사용량 없음

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,2020-08-25 05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,2020-08-25 06,26.1,1.3,83.0,0.0,0.0,NaN,NaN


In [5]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']

for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [6]:
# 시간, 요일, 주말여부(new!) 추가
def time(x):
    return int(x[-2:])
train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

# 평일=0~4, 주말=5~6
def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

# 평일=0, 주말=1
train['weekend']=train['weekday'].apply(lambda x: 0 if x < 4 else 1)
test['weekend']=test['weekday'].apply(lambda x: 0 if x < 4 else 1)

In [7]:
# 기온, 풍속, 습도 등, 기타 결측치를 적당히 1/3, 2/3 수치로 보간해줍니다.
test = test.interpolate(method='values')  
test.head(3)

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday,weekend
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.0,0.0,0.0,0,1,0
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.0,0.0,0.0,1,1,0
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.0,0.0,0.0,2,1,0


In [8]:
train.head(3)

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,time,weekday,weekend
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,0,0


# 모델링

In [9]:
# 학습용set 생성
train.drop('date_time', axis=1, inplace=True)  # 학습에 불필요한 날짜 제거
train_x=train.drop('전력사용량(kWh)', axis=1)  # 문제
train_y=train[['전력사용량(kWh)']]  # 정답

X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=156)

In [10]:
X_train.head(1)

,num,기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,time,weekday,weekend
91868,46,21.0,5.2,80.0,0.0,0.0,1.0,0.0,20,2,0


In [11]:
y_train.head(1)

,전력사용량(kWh)
91868,1609.632


In [12]:
# loss function : SMAPE 정의
# from sklearn.metrics import mean_absolute_error
def smape(true, pred):
    true = np.array(true)  # np.array로 바꿔야 에러 없음
    pred = np.array(pred)
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred)))  # *2 , *100은 상수이므로 생략
SMAPE = make_scorer(smape, greater_is_better=False)  # smape 값이 작아져야하므로 False

In [13]:
# 파라미터 설정, 모델생성 함수
def get_best_params(model, params):
    grid_model = GridSearchCV(
        model,
        param_grid = params,  # 파라미터
        cv=5,  # Kfold : 5
        scoring= SMAPE)  #loss function

    grid_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=100)
    scr = grid_model.best_score_
    print(f'{model.__class__.__name__} 최적 score 값 {scr}')
    return grid_model.best_estimator_

In [14]:
# 파라미터 후보군 설정
# 어떤 파라미터로 하는게 좋을지 고민된다면 고민하는 것들을 리스트 안에 다 넣어보세요 알아서 골라줄겁니다.
# 저는 예시로 learning_rate만 0.1 or 0.01 중 더 좋은걸 골라달라고 했습니다.
params = {}
params['boosting_type'] = ['gbdt']
params['objective'] = ['regression']
params['n_estimators'] = [100]
params['learning_rate'] = [0.1, 0.01]  
params['subsample'] = [1]

In [15]:
# 모델정의
model=LGBMRegressor(params)

In [16]:
# 학습진행
best_lgbm = get_best_params(model, params)
best_lgbm  # learning_rate 0.1 or 0.01 중, 0.01이 더 좋았다고 하는군요. 

[100]	valid_0's l2: 110307	valid_1's l2: 119800
[100]	valid_0's l2: 108222	valid_1's l2: 115531
[100]	valid_0's l2: 109730	valid_1's l2: 117630
[100]	valid_0's l2: 107259	valid_1's l2: 115403
[100]	valid_0's l2: 108007	valid_1's l2: 116569
[100]	valid_0's l2: 1.14647e+06	valid_1's l2: 1.18634e+06
[100]	valid_0's l2: 1.13986e+06	valid_1's l2: 1.1809e+06
[100]	valid_0's l2: 1.14542e+06	valid_1's l2: 1.18557e+06
[100]	valid_0's l2: 1.1387e+06	valid_1's l2: 1.1782e+06
[100]	valid_0's l2: 1.13814e+06	valid_1's l2: 1.17912e+06
[100]	training's l2: 1.13759e+06	valid_1's l2: 1.17786e+06
LGBMRegressor 최적 score 값 -0.30167197091483605


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.01, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1,
              subsample_for_bin=200000, subsample_freq=0)

In [17]:
# 기타 지표로 에러 측정
from sklearn.metrics import mean_squared_error, r2_score
y_pred = best_lgbm.predict(X_train)

mse_score = mean_squared_error(y_train, y_pred)
r2_score = r2_score(y_train, y_pred)
print('MSE:', mse_score)
print('R2 :', r2_score)

MSE: 1137593.8469961581
R2 : 0.7285708061170786


In [18]:
# 모델저장, 로드
from sklearn.externals import joblib
joblib.dump(best_lgbm, 'best_lgbm.pkl')
load_lgbm = joblib.load('best_lgbm.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# 제출용 데이터

In [19]:
# 모델에 넣기 위해 날짜칼럼 제거
test_x = test.drop('date_time', axis=1)

In [20]:
# 모델 예측
submission_y = best_lgbm.predict(test_x)

In [21]:
# submission.csv 생성
test['answer'] = submission_y
test['num_date_time'] = test.apply(lambda x: str(x['num']) +' '+ x['date_time'], axis=1)
submission = test[['num_date_time', 'answer']]
submission.to_csv('submission.csv', index=False)
submission

,num_date_time,answer
0,1 2020-08-25 00,6264.865912
1,1 2020-08-25 01,6264.865912
2,1 2020-08-25 02,6264.865912
3,1 2020-08-25 03,6264.865912
4,1 2020-08-25 04,6264.865912
...,...,...
10075,60 2020-08-31 19,2809.636990
10076,60 2020-08-31 20,2392.383243
10077,60 2020-08-31 21,2343.381185
10078,60 2020-08-31 22,2343.381185
